<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/CM_G12T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 47.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 979 kB 11.0 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 138 

In [38]:
clmns = [
       "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15",

    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(30, data.shape[0]-1):
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        sugg = "sell"
        if data[i][3] > data[i-1][3]:
            sugg = "buy"
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [9]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [10]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1mo")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [39]:
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1095
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240  
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

In [40]:
xTrain

,Open 1,Close 1,Open 2,Close 2,Open 3,Close 3,Open 4,Close 4,Open 5,Close 5,...,Open 11,Close 11,Open 12,Close 12,Open 13,Close 13,Open 14,Close 14,Open 15,Close 15
16546,10.000000,2.274080,-1.623412,-2.784884,-6.107287,-4.033663,-1.975676,-1.713758,-6.656749,-8.127918,...,-7.195787,-7.403410,-8.365078,-8.485829,-9.405799,-9.882289,-9.766316,-9.385384,-5.859703,-4.465849
1830,10.000000,3.785491,5.532223,1.247029,0.030144,1.704605,2.193109,-1.542330,-4.127103,-4.608040,...,-9.533581,-9.739310,-10.000000,-9.872462,-9.725342,-9.724600,-9.771249,-9.543910,-9.219248,-9.310113
13793,9.775972,10.000000,1.995390,-3.993109,-4.481959,-3.605944,1.229197,-0.592338,-1.626573,-6.747904,...,-9.013716,-8.498859,-8.731025,-8.817707,-9.309337,-9.629563,-10.000000,-9.014734,-8.010032,-7.727089
14845,-9.106794,-7.809936,-7.586895,-7.371643,-6.498026,-0.239497,-3.772847,-4.498520,-3.382992,-4.649871,...,-1.910296,1.170525,-1.265402,-1.325445,-3.846804,-5.652483,-1.288697,-1.803332,-10.000000,-8.197427
19602,10.000000,9.015062,1.524705,-4.930095,-6.790551,-3.815384,-4.444158,-7.780165,-6.947878,-6.721419,...,6.966001,-1.158122,-3.887713,-6.817635,-9.761065,-10.000000,-9.677801,-6.170982,-3.809713,-3.651574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16367,-9.471094,-9.651301,-10.000000,-9.178467,-8.586147,-8.260513,-6.411574,-4.599619,-7.915607,-6.396860,...,6.613196,1.604248,1.307613,3.399195,10.000000,-0.029611,1.916942,4.089772,2.660623,7.231824
9609,-7.705703,-8.119668,-4.302256,-2.216567,-1.631303,-7.872160,-9.036305,-10.000000,-8.765329,-8.958449,...,-7.690676,-7.241939,-7.562348,-3.342697,7.859015,10.000000,0.574467,-0.798864,-0.264955,-5.660097
12009,10.000000,8.125643,-3.789297,-3.315297,-8.263081,-7.961149,-4.728640,-5.280558,-3.551215,-7.382176,...,-2.370542,1.243980,-2.843461,-2.948433,-5.229695,-5.499161,-3.775228,-3.249283,6.607327,8.420000
16250,-7.824935,-7.614812,-5.336398,-3.896388,-2.677036,-3.138139,-0.938766,-4.509174,-4.508222,-2.005201,...,-9.343662,-9.332055,-9.583097,-10.000000,-9.623144,-9.681362,-9.635825,-9.168361,-6.981041,-9.159022


In [41]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))


opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 1024)              31744     
                                                                 
 dense_13 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_14 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_15 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_16 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_17 (Dense)            (None, 2)                 2050      
                                                                 
Total params: 4,232,194
Trainable params: 4,232,194
No

In [48]:
model.fit(xTrain,yTrain,epochs=10,batch_size=2048,validation_data=(xTest,yTest))

Epoch 1/10
10/10 [==============================] - 12s 1s/step - loss: 0.0379 - accuracy: 0.9841 - val_loss: 0.2550 - val_accuracy: 0.9555
Epoch 2/10
10/10 [==============================] - 9s 850ms/step - loss: 0.0312 - accuracy: 0.9867 - val_loss: 0.2573 - val_accuracy: 0.9601
Epoch 3/10
10/10 [==============================] - 9s 856ms/step - loss: 0.0278 - accuracy: 0.9884 - val_loss: 0.2701 - val_accuracy: 0.9610
Epoch 4/10
10/10 [==============================] - 8s 849ms/step - loss: 0.0259 - accuracy: 0.9890 - val_loss: 0.2786 - val_accuracy: 0.9624
Epoch 5/10
10/10 [==============================] - 8s 841ms/step - loss: 0.0248 - accuracy: 0.9893 - val_loss: 0.2857 - val_accuracy: 0.9624
Epoch 6/10
10/10 [==============================] - 8s 831ms/step - loss: 0.0241 - accuracy: 0.9896 - val_loss: 0.2923 - val_accuracy: 0.9620
Epoch 7/10
10/10 [==============================] - 8s 822ms/step - loss: 0.0234 - accuracy: 0.9897 - val_loss: 0.2994 - val_accuracy: 0.9620
Epoch 8/

In [ ]:
model.save("CMC_G115_9538.h5")

In [49]:
yf.download("btc-usd",period="max",interval="1mo")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-10-01,387.427002,411.697998,289.295990,338.321014,338.321014,902994450
2014-11-01,338.649994,457.092987,320.626007,378.046997,378.046997,659733360
2014-12-01,378.248993,384.037994,304.231995,320.192993,320.192993,553102310
2015-01-01,320.434998,320.434998,171.509995,217.464005,217.464005,1098811912
2015-02-01,216.867004,265.610992,212.014999,254.263000,254.263000,711518700
...,...,...,...,...,...,...
2022-02-01,38481.765625,45661.171875,34459.218750,43193.234375,43193.234375,671335993325
2022-03-01,43194.503906,48086.835938,37260.203125,45538.675781,45538.675781,830943838435
2022-04-01,45554.164062,47313.476562,37697.941406,37714.875000,37714.875000,830115888649


In [55]:
def process_for_prediction(data):
        i = -2
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [57]:
make_prediction("btc-usd","max","1mo")

[*********************100%***********************]  1 of 1 completed


array([[7.5575709e-04, 9.9966586e-01]], dtype=float32)

In [52]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSDt',exchange='binance',interval=Interval.in_monthly,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [53]:
make_prediction("s","x")

array([[8.200407e-04, 9.996339e-01]], dtype=float32)

In [ ]:
tv = TvDatafeed()
tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_monthly,n_bars=100)



do you want to install chromedriver automatically?? y/n	n


you are using nologin method, data you access may be limited


,symbol,open,high,low,close,volume
datetime,,,,,,
2014-02-01,bitstamp:btcUSD,803.00,827.38,400.00,550.10,824911.000000
2014-03-01,bitstamp:btcUSD,551.80,710.00,436.00,454.83,491824.650960
2014-04-01,bitstamp:btcUSD,454.81,548.00,339.79,448.85,515027.729378
2014-05-01,bitstamp:btcUSD,449.00,629.40,420.27,627.80,300028.000000
2014-06-01,bitstamp:btcUSD,627.80,683.26,538.38,641.11,288061.000000
...,...,...,...,...,...,...
2022-01-01,bitstamp:btcUSD,46230.00,47989.00,32950.72,38491.92,71879.817236
2022-02-01,bitstamp:btcUSD,38483.56,45850.00,34324.05,43178.98,57987.188733
2022-03-01,bitstamp:btcUSD,43221.71,48234.00,37169.52,45517.27,57990.054358
